In [ ]:
%matplotlib inline

In [ ]:
#for wider display
#from IPython.core.display import display, HTML
#display(HTML("<style>.container { width:70% !important; }</style>"))

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from pandas.plotting import scatter_matrix

In [ ]:
#matplotlib style
plt.style.use('fivethirtyeight')

In [ ]:
def profit(rental,pred):
    return (rental*df.loc['2012', ['cnt',pred]].min(axis=1)-2*df.loc['2012', pred]).sum()

In [ ]:
def profit_ratio(rental,pred,):
    return (rental*df.loc['2012', ['cnt',pred]].min(axis=1)-2*df.loc['2012', pred]).sum()/(2*df.loc['2012', pred].sum())

# Data Selection and Preprocessing	
## Data handling and derived variables
Data was import as dataframe and date was used as index.'weekday', 'holiday' and 'temp' were dropped for 'workingday' and 'atemp' contained the same or better infomation.
Because some holidays eg. Christmas has longer effect, days around were also changed as holiday. 'disaster' column was add to indicate extreme weather condition such like storm, tornado and hurrican.
To reduce the effect of nature outlier, columns of 'casual', 'registered' and 'cnt' were transformed into logarithm as 'casual_log' and 'registered_log'. 'casual_log' and 'registered_log' were used as model target variable and were modeled separately. The predicted values were transformed back to normal and added up as predicted 'cnt'.

In [ ]:
#read data from day.csv with date as index and drop extra columns
df = pd.read_csv('day.csv',index_col='dteday', parse_dates=True).drop(['weekday','holiday','temp'], axis=1) #weekday 可能抛早了
df.index.name = 'date'

In [ ]:
# tax day
df.loc['2011-4-15', "workingday"] = 1
df.loc['2012-4-16', "workingday"] = 1
# thanksgiving friday
df.loc['2011-11-25', "workingday"] = 0
df.loc['2012-11-23', "workingday"] = 0

# Christmas
df.loc['2011-12-24':'2011-12-26', "workingday"] = 0
df.loc['2012-12-24':'2011-12-26', "workingday"] = 0

df['disaster'] = 0

#storms
df.loc['2012-5-21', "workingday"] = 0
df.loc['2012-5-21', "disaster"] = 1
#tornado
df.loc['2012-6-1', "workingday"] = 0
df.loc['2012-6-1', "disaster"] = 1
#Hurricane Sandy
df.loc['2012-10-29':'2012-10-30', "workingday"] = 0
df.loc['2012-10-29':'2012-10-30', "disaster"] = 1

In [ ]:
# add 2 new columns with log to reduce the effect of nature outlier, +1 to avoid -∞ and make all value ≥ 0
for col in ['casual', 'registered']:
    df['%s_log' % col] = np.log(df[col] + 1)

In [ ]:
def predict_on_test_set(model, x_cols):
    casual_model = model.fit(train[x_cols], train['casual_log'])
    y_pred_cas = casual_model.predict(test[x_cols])
    y_pred_cas = np.exp(y_pred_cas) - 1
    registered_model = model.fit(train[x_cols], train['registered_log'])
    y_pred_reg = registered_model.predict(test[x_cols])
    y_pred_reg = np.exp(y_pred_reg) - 1
    return y_pred_cas + y_pred_reg

##  Model input variables
Model input variables were including 'weathersit', 'atemp', 'hum','windspeed', 'workingday', 'season', 'yr','mnth','instant'and 'disaster'. All varibles were treated as numeric data.

In [ ]:
rf_cols = [
    'weathersit', 'atemp', 'hum','windspeed',
    'workingday', 'season',  
    'yr','mnth','instant',
    "disaster"
    ]

In [ ]:
gbm_cols = [
    'weathersit', 'atemp', 'hum', 'windspeed',
    'workingday', 'season',
    'yr','mnth','instant',
    "disaster"
    ]

In [ ]:
df.info()

## Test and training set splitting
Test set is 'cnt' value for day in 2012. For each day, one model was built to make prediction more accurate. Training set was 12 month rolling window excluding the current day or all the data before the current day.

# Modeling Building and Testing
##  Modeling algorithms
Gradient Boosting and Random Forest was used for modeling.
For Random Forest the parameters of 'n_estimators': 1000, 'max_depth': 15, 'random_state': 0, 'min_samples_split' : 5, 'n_jobs': -1 were used and take weight of 0.2.
For Gradient Boosting the parameters of 'n_estimators': 150, 'max_depth': 5, 'random_state': 0, 'min_samples_leaf' : 10, 'learning_rate': 0.1, 'subsample': 0.7, 'loss': 'ls' were used and take the weight of 0.8.

In [ ]:
def pred():
    rf_model = RandomForestRegressor(**rf_params)
    rf_pred = predict_on_test_set(rf_model, rf_cols)
    gbm_model = GradientBoostingRegressor(**gbm_params)
    gbm_pred = predict_on_test_set(gbm_model, gbm_cols)
    y_pred = np.round(0.2*rf_pred + 0.8*gbm_pred)
    return y_pred

In [ ]:
rf_params = {'n_estimators': 1000, 'max_depth': 15, 'random_state': 0, 'min_samples_split' : 5, 'n_jobs': -1}

In [ ]:
gbm_params = {'n_estimators': 150, 'max_depth': 5, 'random_state': 0, 'min_samples_leaf' : 10, 'learning_rate': 0.1, 'subsample': 0.7, 'loss': 'ls'}


In [ ]:
#1 year rolling
test_slicing = pd.to_datetime('2012-1-1')
for n in range(366):
    if n == 0:
        train = df['2011-1-1':'2011-12-30']
    else:
        train = df.loc[pd.to_datetime('2011-1-1')+pd.DateOffset(days= n-1):pd.to_datetime('2012-1-1')+pd.DateOffset(days= n -2)]
    test = df.loc[[test_slicing+pd.DateOffset(days=n)],:]
    df.loc[test_slicing+pd.DateOffset(days=n),'pred_cnt_rolling_1year'] = pred()[0]
    

In [ ]:
rentals = pd.Series([3,8,2.2])

In [ ]:
profit_rolling_1year = rentals.apply(profit, args=('pred_cnt_rolling_1year',))
profit_ratio_rolling_1year = rentals.apply(profit_ratio, args=('pred_cnt_rolling_1year',))

In [ ]:
#all data rolling
test_slicing = pd.to_datetime('2012-1-1')
for n in range(366):
    train = df.loc[pd.to_datetime('2011-1-1'):pd.to_datetime('2012-1-1')+pd.DateOffset(days= n -2)]
    test = df.loc[[test_slicing+pd.DateOffset(days=n)],:]
    df.loc[test_slicing+pd.DateOffset(days=n),'pred_cnt_rolling_all'] = pred()[0]

In [ ]:
profit_rolling_all = rentals.apply(profit, args=('pred_cnt_rolling_all',))
profit_ratio_rolling_all = rentals.apply(profit_ratio, args=('pred_cnt_rolling_all',))

In [ ]:
df.to_csv('predictions.csv')

## Testing
这里不太清楚是分别test还是ensemble后的模型

# Business	Performance

In [ ]:
#2011
train = df['2011']
test = df['2012']
df.loc['2012','pred_cnt_with_2011'] = pred()

In [ ]:
profit_with_2011 = rentals.apply(profit, args=('pred_cnt_with_2011',))
profit_ratio_with_2011 = rentals.apply(profit_ratio, args=('pred_cnt_with_2011',))

In [ ]:
#perfect
profit_perfect = rentals.apply(profit, args=('cnt',))
profit_ratio_perfect = rentals.apply(profit_ratio, args=('cnt',))

In [ ]:
#default
df.loc['2012-1-1':'2012-12-31','pred_cnt_default'] = np.array(df.loc['2011-12-30':'2012-12-29','cnt'])

In [ ]:
profit_default = rentals.apply(profit, args=('pred_cnt_default',))
profit_ratio_default = rentals.apply(profit_ratio, args=('pred_cnt_default',))

##  Profit for 2012
Our model had a total profit 1734647 for the year of 2012.  The default prediction had a total profit 1442972 for the year of 2012

In [ ]:
profit = pd.DataFrame({'perfect':profit_perfect,'1year rolling':profit_rolling_1year, 'all before':profit_rolling_all, '2011 only':profit_with_2011, 'defaut':profit_default})

In [ ]:
profit.index = rentals

In [ ]:
profit

In [ ]:
#sub default
sub_with_default = profit.sub(profit['defaut'], axis=0)
sub_with_default

In [ ]:
#div perfect
div_with_default = profit.div(profit['perfect'], axis=0)
div_with_default

## Profit expressed as a percentage of total expenditure
Our model has the profit/cost ratio of 0.4436 for the year of 2012. The default prediction has the profit/cost ratio of 0.3519 for the year of 2012

In [ ]:
profit_ratio = pd.DataFrame({'perfect':profit_ratio_perfect,'1year':profit_ratio_rolling_1year, 'all before':profit_ratio_rolling_all, '2011 only':profit_ratio_with_2011, 'defaut':profit_ratio_default})
profit_ratio.index = rentals

In [ ]:
profit_ratio

## Performance with rental variation
Our prediction model is always better than the default model. 

## Performance with season or other variation
model performance correlated with season, month and working days

In [ ]:
def profit_perfect_ratio(df, pred,rental):
    return (rental*df.loc[:, ['cnt',pred]].min(axis=1)-2*df.loc[:, pred]).sum()/((rental-2)*df.loc[:, pred].sum())

In [ ]:
df.loc['2012',['season','cnt','pred_cnt_rolling_1year']].groupby('season').apply(profit_perfect_ratio,pred = 'pred_cnt_rolling_1year',rental = 3)

In [ ]:
df.loc['2012',['mnth','cnt','pred_cnt_rolling_1year']].groupby('mnth').apply(profit_perfect_ratio,pred = 'pred_cnt_rolling_1year',rental = 3)

In [ ]:
df.loc['2012',['workingday','cnt','pred_cnt_rolling_1year']].groupby('workingday').apply(profit_perfect_ratio,pred = 'pred_cnt_rolling_1year',rental = 3)

## Performance with age variation
Our model performance did not decreases with the age of the model. 年底的可能是圣诞新年长假影响。10月份的应该是Hurricane Sandy。

In [ ]:
df2 = df.loc['2012',['cnt','pred_cnt_rolling_1year']]
df2['weekofyear'] = df2.index.weekofyear
df2.groupby('weekofyear').apply(profit_perfect_ratio,pred = 'pred_cnt_rolling_1year',rental = 3).plot()

## Performance with train set variation
The model generated from the 18month training set (Jan’11 to Jun’12) give higher profit than the model built from the 12month training set when tested on the test period Jul’12 to Dec’12(905824.0, 898264.0). Although the difference is relative small.

In [ ]:
#train with 18m (2011-1 - 2012-6)
train = df['2011-1':'2012-6']
test = df['2012-7':'2012-12']
df.loc['2012-7':'2012-12','pred_cnt_with_18m'] = pred()

In [ ]:
profit_with_18m_in_6m = (3*df.loc['2012-7-1':'2012-12-31', ['cnt','pred_cnt_with_18m']].min(axis=1)-2*df['pred_cnt_with_18m']).sum()
profit_with_18m_in_6m

In [ ]:
#train with 12m (2011-7-1 - 2012-6-30)
train = df['2011-7':'2012-6']
test = df['2012-7':'2012-12']
df.loc['2012-7':'2012-12','pred_cnt_with_12m'] = pred()

In [ ]:
profit_with_12m_in_6m = (3*df.loc['2012-7':'2012-12', ['cnt','pred_cnt_with_12m']].min(axis=1)-2*df['pred_cnt_with_12m']).sum()
profit_with_12m_in_6m

## Performance with balancing data
After balancing the data, the evidence that gave higher performance could be seen (1033056.2 vs 905824.0).

In [ ]:
#train with 18m (2011-1 - 2012-6) wtih down sampling
n=20
pred_cnt_with_18m_wtih_down_sampling = np.zeros(n)
for i in range(n):
    train = pd.concat([pd.concat([df['2011-1':'2011-6'],df['2012-1':'2012-6']]).sample(frac=(365-184)/363),df['2012-7':'2012-12']])
    test = df['2012-7':'2012-12']
    df.loc['2012-7':'2012-12','pred_cnt_with_18m_wtih_down_sampling'] = pred()
    pred_cnt_with_18m_wtih_down_sampling[i] = (3*df.loc['2012-7-1':'2012-12-31', ['cnt','pred_cnt_with_18m_wtih_down_sampling']].min(axis=1)-2*df['pred_cnt_with_18m_wtih_down_sampling']).sum()

In [ ]:
pred_cnt_with_18m_wtih_down_sampling

In [ ]:
pred_cnt_with_18m_wtih_down_sampling.mean()
